In [68]:
import pandas as pd
#import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression as LR
from xgboost import XGBClassifier
import statistics
import shap
import numpy.core.multiarray


#return data before number
def beforenum(data):
    data=str(data)
    for i in range(len(data)):
        if unicode(str(data[i])).isnumeric():
            return data[:i]
    return data


#return data before number
def onlyint(data):
    data=str(data)
    for i in range(len(data)):
        if ~unicode(str(data[i])).isnumeric():
            #print ('yes')
            return data[:i]
    return data

#return float before text
def onlyfloat(data):
    data=str(data)
    for i in range(len(data)):
        if ~unicode(str(data[i])).isnumeric() and str(data[i])!='.':
            #print ('yes')
            return data[:i]
        i=i+1
    return data


#normalize the dataframe
def normalizedataframe(df):

    from sklearn import preprocessing
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(df.values)
    return pd.DataFrame(x_scaled,columns=df.columns.values)

# if one row is a category and another corresponding row is the value turns that category into a columna and puts the data in it
# example lab test| value| patient
#         flu     | 3    |1
#         fever   | 2   |1
#         flu    | 4    |2
#   patient| flu| fever
#    1| 3| 2
#    1| 4 | null
def catvalueget(data,catcol,valcat,keep,joinonleft,joinonright,minc):
    test=data
    names=data[catcol].value_counts().index
    l=len(names)
    #returns only categories with at least x amount of unique values,
    l=mincount(data,catcol,minc)
    i=0
    #creates column with 
    while i<l:
        value=names[i]
        value=str(value)
        #test[value]=0
        test2=test[test[catcol]==value]
        test2[value]=0
        keep=list(keep)
        keep.append(valcat)
        test2=test2[keep]
        test2=test2.rename(columns={valcat:value})
        test = pd.merge(test, test2,  how='left', left_on=joinonleft, right_on = joinonright)
        test = test.loc[:,~test.columns.duplicated()]
        i=i+1
    test = test.loc[:,~test.columns.duplicated()]
    return test





    
# remove all columns that can't be turned into float
def removenonnumcols(data):
    cols=data.columns.values
    for col in cols:
        try:
            data[col]=data[col].astype(float)
        except:
            data=data.drop(columns=[col])
        i=i+1
    data=data.fillna(-1)
    return data
        

#return place that value count is lower than min count
def mincount(data,col,Min):
    lis=data[col].value_counts()
    i=0
    l=len(lis)
    while i<l:
        amount=float(lis[i])
        if amount<Min:
            return i
        i=i+1
        
    i=i-1
    return i

# fits the data to logistic regression to get weights of features and see how they perform
# write the feature weights to file for you to view
# returns weights * coef_ and feature name
def simplefeatureselection(data,predict,targetlist,weightfile):
    if len(targetlist)!=len(predict.value_counts().index):
        print ('Targetlist is Length: '+str(len(targetlist))+" But the length should be: "+str(len(predict.value_counts().index)))
    clf=LR()
    clf=clf.fit(data,predict)
    featureweights=np.std(data.values, 0)*clf.coef_
    temp = pd.DataFrame(featureweights, columns=data.columns.values)
    temp.T.to_csv(weightfile)
    temp['Target']=targetlist
    return temp

# gets top amount of features based on average weight for that feature, and returns those features to limit the amount of features
def rankingfeatures(data,target,top):
    temp=data
    temp2=temp.set_index(target)
    tempabs=abs(temp2)
    tempabs=tempabs.T
    l=len(tempabs.columns.values)
    i=0
    topamount=top
    while i<l:
        if True:
            tempabs=tempabs.sort_values(by=tempabs.columns.values[i],ascending=False)
            order= (tempabs.index[:topamount])    
            temptemp=temp2[temp2.index==tempabs.columns.values[i]].reset_index()
            temptemp= (temptemp[order])
            temptempval=temptemp.values
            if i==0:
                extradataframe=pd.DataFrame({str(tempabs.columns.values[i])+'_Weights':temptempval[0],str(tempabs.columns.values[i])+'_Names':order})
            if True:
                extradataframe[str(tempabs.columns.values[i])+'_Weights']=list(temptempval[0])
                extradataframe[str(tempabs.columns.values[i])+'_Names']=order

        i=i+1
    return extradataframe


#returns important features based on top x amount
def getfeaturesback(data,predict,targetlist,weightfile,top):
    try:
        predict2=predict
        predict=data[predict]
        data=data.drop(columns=predict2)
    except:
        True
    if True:
        temp=simplefeatureselection(data,predict,targetlist,weightfile)
        temp=rankingfeatures(temp,'Target',top)
    return temp


# get columns in the data and how often they occur with the specific ending
def commonfeatures(data,ending='_Names'):
    data = data.filter(regex=ending)
    cols=data.columns.values
    items=[]
    for col in cols:
        items=items+(list(data[col].values[:]))
    myset = list(set(li))
    countlist=[]
    for item in myset:
        countlist.append(items.count(item))
        i=i+1
    df=pd.DataFrame({'COL':myset,'Count':countlist})
    return df


#write the performance of the XGBoost model
def writeperformance(data,predicted,test,file):
    try:
        predict2=predicted
        predicted=data[predicted]
        data=data.drop(columns=predict2)
    except:
        True
    data['Predictval']=predicted
    f=open(file,'w')
    f.write("CUTOFF\tTP\tFP\tTN\tFN\n")
    cutoff=0
    while cutoff<1:
        mask=data['Predictval']>=cutoff
        data["Value"]=0
        data.loc[mask,'Value']=1
        confusionmatrix=confusion_matrix(data[test],data['Value'])
        tn = confusionmatrix [0][0]
        fn = confusionmatrix [1][0]
        tp = confusionmatrix [1][1]
        fp = confusionmatrix [0][1]
        f.write(str(cutoff)+"\t"+str(tp)+"\t"+str(fp)+"\t"+str(tn)+"\t"+str(fn)+"\n")
        cutoff=cutoff+.01
    f.close()

    
#return list of features that appear x amount of times or greater in the list
def minocc(coldata,cutoff,colcol='COL',cutoffcol='Count'):
    coldata=coldata[coldata[cutoffcol]>cutoff]
    return list(coldata[colcol].value_counts().index)




#trains the XGBoost model
def trainxgboost(data,cols,predict,md=3,ne=1000,g=5,lr=.1):
    try:
        predict2=predict
        predict=data[predict]
        data=data.drop(columns=predict2)
    except:
        True
    
    clf=XGBClassifier(max_depth=md,n_estimators=ne,gamma=g,learning_rate=lr)
    return clf.fit(data[cols],predict)
    

    
#split train test data to train and predict on on an XGBoost Model
def splittest(data,cols,predict,file,md=3,ne=1000,g=5,lr=.1):
    try:
        predict2=predict
        predict=data[predict]
        data=data.drop(columns=predict2)
    except:
        True 
    X_train, X_test, y_train, y_test = train_test_split(
        data, predict, test_size=0.33, random_state=42)
    clf=trainxgboost(X_train,cols,y_train)
    
    predict=clf.predict_proba(X_test[clf.get_booster().feature_names])
    X_test['Actual']=y_test
    writeperformance(X_test,predict[:,1],'Actual',file)

    
    
#rturn plot of feature importance
def Graphfeatures(clf,x):
    import numpy.core.multiarray
    import numpy
    temp=clf.booster().get_score(importance_type='gain')
    explainer=shap.TreeExplainer(clf)
    shap_values = explainer.shap_values(x)
    pic=shap.summary_plot(shap_values, x,feature_names=xvec.get_feature_names(),max_display=200)
    print(pic)


    
# predict binary classification after cleaning data
def completebasiccyclebinary(data,predict,targetlist,top,mincount,file,weightfile,md=3,ne=1000,g=5,lr=.1):

    testing= (getfeaturesback(data,predict,targetlist,weightfile,top))
    testing=commonfeatures(testing)
    cols=minocc(testing,mincount)
    splittest(data,cols,predict,file,md,ne,g,lr)
    
    return (True)


#find groupings of values making buckets between min and max amounts of buckets
def groupingsten(data,col,maxbucket,minbucket):
    testnum=data[col].quantile(.9)-data[col].quantile(.1)
    l=len(str(testnum))
    divide=10/(pow(10,l))
    while (testnum/divide>=maxbucket or testnum/divide<=minbucket):
        divide=divide*10
    return divide



#divides into groups of of second nearest 10 so if 200 then 10-20,20-30,120-130, if 10 then 1 2 3 4 5, if 8.0 then .1,.2,.3,.4,7.7-7.8
def standarddivide(data,col,target):
    data=data[[col,target]]
    data[col]=data[col].astype(float)
    data[target]=data[target].astype(float)
    # gets groupings between number of buckets
    divide=groupingsten(data,col,100,10)
    mini=data[col].min()
    maxi=data[col].max()
    while mini<maxi+divide:
        data[str(mini)+"-"+str(mini+divide)]=0
        mask=((data[col]>=mini) & (data[col]<=mini+divide))
        data.loc[mask,str(mini)+"-"+str(mini+divide)]=1
        mini=mini+divide
    data=data.drop(columns=col)
    score=corrolationtopaverage(data,target,.3)
    return data,score

 # tries all amount of bins using pandas qcut from minbucket to maxbucket amount and scores the correlation of the top percent
# finds the best amount of buckets for that continuous column    
def bestqcut(data,col,target,minbucket,maxbucket,percent):
    #print (data)
    data=data[[col,target]]
    bestbuckets=0
    bestscore=0
    while minbucket<maxbucket:
        temptest=list(set(list(pd.qcut(data[col],q=minbucket))))
        l=len(temptest)
        i=0
        while i<l:
            loc=str(temptest[i]).find(',')
        
            minc= float((str(temptest[i])[1:loc]))
            maxc= float((str(temptest[i])[loc+2:-1]))
            mask=((data[col]>=minc) & (data[col]<=maxc))
            data[str(minc)+'_'+str(maxc)]=0
            data.loc[mask,str(minc)+'_'+str(maxc)]=1
            
            i=i+1
        data2=data.drop(columns=col)
        score=corrolationtopaverage(data2,target,percent)
        if score>bestscore:
            bestscore=score
            bestbuckets=minbucket
        minbucket=minbucket+1
    return bestscore,bestbuckets
        

 # tries all amount of bins using pandas quantile from minbucket to maxbucket amount and scores the correlation of the top percent
# finds the best amount of buckets for that continuous column        
def bestquant(data,col,target,minbucket,maxbucket,percent):
    data=data[[col,target]]
    bestbucket=0
    bestscore=0
    while minbucket<maxbucket:
        moving=1/minbucket
        minc=0
        while minc<1:
            if minc+moving>=100:
                data[str(minc)+'_1']=0
                mask=data[col]>data[col].quantile(minc)
                data.loc[mask,str(minc)+'_1']=1
            
            data[str(minc)+'_'+str(minc+moving)]=0
            mask=((data[col]>=data[col].quantile(minc)) & (data[col]<=data[col].quantile(minc+moving)))
            data.loc[str(minc)+'_'+str(minc+moving)]=1
            minc=minc+1
        data2=data.drop(columns=col)
        score=corrolationtopaverage(data2,target,percent)
        if score>bestscore:
            bestscore=score
            bestbuckets=minbucket
        minbucket=minbucket+1
    return bestscore,bestbuckets
                
            
            
    
 # tries all amount of bins using pandas.cut from minbucket to maxbucket amount and scores the correlation of the top percent
# finds the best amount of cuts for that continuous column
def bestcut(data,col,target,minbucket,maxbucket,percent):
    data=data[[col,target]]
    bestbuckets=0
    bestscore=0
    while minbucket<maxbucket:
        temptest=list(set(list(pd.cut(data[col],bins=minbucket))))
        l=len(temptest)
        i=0
        while i<l:
            loc=str(temptest[i]).find(',')
        
            minc= float((str(temptest[i])[1:loc]))
            maxc= float((str(temptest[i])[loc+2:-1]))
            mask=((data[col]>=minc) & (data[col]<=maxc))
            data[str(minc)+'_'+str(maxc)]=0
            data.loc[mask,str(minc)+'_'+str(maxc)]=1
            
            i=i+1
        data2=data.drop(columns=col)
        score=corrolationtopaverage(data2,target,percent)
        if score>bestscore:
            bestscore=score
            bestbuckets=minbucket
        minbucket=minbucket+1
    return bestscore,bestbuckets


#return the average correlation for the top percentage of features
def corrolationtopaverage(data,target,percent):
    data=data.corr()
    cor=list(abs(data[target][:].values)).sort(reverse=False).pop().sort(reverse=True)
    if int(len(cor)*percent) <1:
        amount=1
    else:
        amount=int(len(cor)*percent)
    return ( statistics.mean(cor[:amount]))


# after finding the best cut the column into those categories
def cutting(data,col,cuttype,cutbucket):

    if cuttype=='QCUT' or cuttype=='CUT':
        if cuttype=='QCUT':
            temptest=list(set(list(pd.qcut(data[col],q=cutbucket))))
        else:
            temptest=list(set(list(pd.cut(data[col],bins=cutbucket))))
        while cols in temptest:
            loc=str(cols).find(',')
            minc= float((str(cols)[1:loc]))
            maxc= float((str(cols)[loc+2:-1]))
            mask=((data[col]>=minc) & (data[col]<=maxc))
            data[str(col)+'_'+str(minc)+'_'+str(maxc)]=0
            data.loc[mask,str(col)+'_'+str(minc)+'_'+str(maxc)]=1
            i=i+1
        data=data.drop(columns=col)
        return data
    if cuttype=='QUANT':
        
        moving=1/cutbucket
        minc=0
        while minc<1:
            if minc+moving>=100:
                data[str(col)+'_'+str(minc)+'_1']=0
                mask=data[col]>data[col].quantile(minc)
                data.loc[mask,str(col)+'_'+str(minc)+'_1']=1
            data[str(col)+'_'+str(minc)+'_'+str(minc+moving)]=0
            mask=((data[col]>=data[col].quantile(minc)) & (data[col]<=data[col].quintile(minc+moving)))
            data.loc[mask,str(col)+'_'+str(minc)+'_'+str(minc+moving)]=1
            minc=minc+1
        data=data.drop(columns=col)
        return data

    
#testing the best way to cut the column, and then decides how many categories to cut it into
# if category wont be better for continuous  then keeps it as is
def bestcutselect(data,col,target,minbucket,maxbucket,percent):
    #finds best  amount based on correlation with target
    qcutscore,qcutbucket=bestqcut(data,col,target,minbucket,maxbucket,percent)
    quantscore,quantbucket=bestquant(data,col,target,minbucket,maxbucket,percent)
    cutscore,cutbucket=bestcut(data,col,target,minbucket,maxbucket,percent)
    bestscore=corrolationtopaverage(data[[col,target]],target,percent)
    temp,standardscore=standarddivide(data[[col,target]],col,target)
    # which best cut amount worked best
    if ((qcutscore>=quantscore) and (qcutscore>=cutscore) and (qcutscore>bestscore)and (qcutscore>=standardscore)):
        return cutting(data,col,'QCUT',qcutbucket)
    if ((quantscore>=qcutscore) and (quantscore>=cutscore)and (quantscore>bestscore)and (quantscore>=standardscore)):
        return cutting(data,col,'QUANT',qcutbucket)
    if ((cutscore>=qcutscore) and (cutscore>=quantscore)and (cutscore>bestscore) and (cutscore>=standardscore)):
        return cutting(data,col,'CUT',qcutbucket)
    if ((standardscore>=qcutscore)and (standardscore>=cutscore) and (standardscore>=quantscore)and (standardscore>bestscore)):
        temp,standscore=standarddivide(data[[col,target]],col,target)
        return temp
    else:
        return data


#Calculate average length of text
def averageLen(lst):
    lengths = [len(str(i)) for i in lst]
    return 0 if len(lengths) == 0 else (float(sum(lengths)) / len(lengths))


#figures out if column is a category or string column
def catorstring(data,catlist,minlength=30):
    strlist=[]
    for cat in catlist:
        # this is used based on average length of text to tell if string or category
        #if averageLen(list(data[cat].value_counts().index))>minlength:
        #this is ued based on amount of unique values in the column to tell if free text or category
        if len(data[cat].value_counts().index)>minlength:
            strlist.append(cat)
            catlist.remove(cat)
    return catlist,strlist


#figures out if its really a number or text/category column
def intorcatclean(data,cols,intlist=[],catlist=[]):
    for col in cols:
        # if after after keeping numbers only there are more than 3 unique values then actually number
        if len(data[col].astype(str).str.replace("\d+.", "").str.replace(".",'').value_counts().index) > 3:
            catlist.append(col)
        else :
            intlist.append(col)
    return intlist,catlist



# add columns to data based on text vectorizer sent, TFIDF and Count work
def vectorizertopandas(data,cols,vectorizer,addon=''):
    xvec=vectorizer.fit(data[cols])
    xll=xvec.transform(data[cols])
    #vectorizer and turn vector data into dense dataframe
    xll2=pd.DataFrame(xll.todense(),columns=xvec.get_feature_names())
    # add columns based on clean words/text
    xll2.columns = [(col).encode('utf-8').strip() + (addon).encode('utf-8').strip() for col in xll2.columns]
    #add columns to original data
    data=pd.concat([data,xll2],axis=1)
    return(data.drop(columns=cols))


#transform columns based on what is found
def fixdatatypes(data,catcols,intcols,strcols,vectorizer):
    #create dummies based off of category columns
    data=pd.get_dummies(data, prefix=catcols, columns=catcols)
    #transforms int columns into only numbers, cleans up dirty data
    for col in intcols:
        data[col]=data[col].astype(str)
        data[col]=data[col].str.replace("[^\d\.]", "")
        data[col]=data[col].str.strip()
        mask=data[col]==''
        data.loc[mask,col]=-1
        data[col]=data[col].str.strip()
        data[col]=data[col].astype(float,errors='ignore')
    # creates word columns from text
    for col in strcols:     
        data=vectorizertopandas(data,col,vectorizer,str(col+'_'))
    return data


def simplecompleterun(data,target,outputfile,weightfile,targetlist=['Positive'],top=20,mincount=1,minbucket=3,maxbucket=10,percent=.30,vectorizer=CountVectorizer( ngram_range=(1,4), max_df=.9999,min_df=.001)):
    
    #list of number columns
    intlist=[]
    #list of text columns
    catlist=[]
    #make dataframe without the target column
    data2=data.drop(columns=target)
    #text or number
    intlist,catlist=intorcatclean(data,list(data2.columns.values))
    #finds out if text columns is category or string category
    catlist,strlist=catorstring(data,catlist)
    
    #clean up number columns and create columns based off free text
    data=fixdatatypes(data,catlist,intlist,strlist,vectorizer)
    l=len(intlist)
    i=0
    while i<l:
        
        data=bestcutselect(data,intlist[i],target,minbucket,maxbucket,percent)
        i=i+1
    completebasiccyclebinary(data,target,targetlist,top,mincount,outputfile,weightfile)
    print ('Finished')
    #print (data.head())
    #print (data.columns.values)

ModuleNotFoundError: No module named 'xgboost'

0    1.0
1    2.0
2    3.3
3      4
4      5
Name: Mix, dtype: object